# Geometric Deep Learning Project - Towards Sparse Hierarchical Graph Classifiers

*Alessia Ruggeri*

### Implementation of the paper *Towards Sparse Hierarchical Graph Classifiers* tested on Enzymes, Proteins and D&D biological datasets using Tensorflow 2.0.

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import os,sys,inspect
import networkx as nx
import numpy as np
import scipy
import matplotlib.pyplot as plt
import time

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Dense, Flatten, Activation, Dropout, LeakyReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Model

from load_data import read_graphfile

In [0]:
### Unzip datasets folders

# !unzip -o data.zip
!unzip -o data_ENZYMES.zip
!unzip -o data_DD.zip
!unzip -o data_PROTEINS.zip
# !unzip -o data_COLLAB.zip

In [0]:
### Load datasets from data.zip

# print("\nLoading ENZYMES...")
# graphs_ENZYMES = read_graphfile(datadir="data", dataname="ENZYMES", max_nodes=None)

# print("\nLoading DD...")
# graphs_DD = read_graphfile(datadir="data", dataname="DD", max_nodes=None)

# print("\nLoading PROTEINS...")
# graphs_PROTEINS = read_graphfile(datadir="data", dataname="PROTEINS", max_nodes=None)

# print("\nLoading COLLAB...")
# graphs_COLLAB = read_graphfile(datadir="data", dataname="COLLAB", max_nodes=None)


### Load datasets from data_DATASET.zip

print("\nLoading ENZYMES...")
graphs_ENZYMES = read_graphfile(datadir="data_ENZYMES", dataname="ENZYMES", max_nodes=None)

print("\nLoading DD...")
graphs_DD = read_graphfile(datadir="data_DD", dataname="DD", max_nodes=None)

print("\nLoading PROTEINS...")
graphs_PROTEINS = read_graphfile(datadir="data_PROTEINS", dataname="PROTEINS", max_nodes=None)

# print("\nLoading COLLAB...")
# graphs_COLLAB = read_graphfile(datadir="data_COLLAB", dataname="COLLAB", max_nodes=None)